In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# L1 ImgIn shape=(?, 28, 28, 1)
# filter size = 3x3  filter Cnt = 32
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))

#  Conv  ->  (?, 28, 28, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

#  Pool  ->  (?, 14, 14, 32)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L1)

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))

#  Conv  ->  (?, 14, 14, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)

#  Pool  ->  (?, 7, 7, 64)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L2)

# fully-connected layer를 위해 값 펼치기
L2 = tf.reshape(L2, [-1, 7 * 7 * 64])
print(L2)

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)


In [5]:
# Fully Connected layer

learning_rate = 0.01
training_epochs = 15
batch_size = 100

W3 = tf.get_variable("W5", shape=[7*7*64, 10], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

hypothesis = tf.matmul(L2, W3) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Learning start...')

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y:batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
print('Learning Finished!!')

correct_prediction = tf.equal(tf.argmax(hypothesis, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Learning start...
Epoch: 0001 cost = 0.204641714
Epoch: 0002 cost = 0.060247025
Epoch: 0003 cost = 0.046074603
Epoch: 0004 cost = 0.040289701
Epoch: 0005 cost = 0.036208712
Epoch: 0006 cost = 0.030121351
Epoch: 0007 cost = 0.031684948
Epoch: 0008 cost = 0.031351824
Epoch: 0009 cost = 0.025767706
Epoch: 0010 cost = 0.020760647
Epoch: 0011 cost = 0.026005771
Epoch: 0012 cost = 0.023467062
Epoch: 0013 cost = 0.023542508
Epoch: 0014 cost = 0.019875769
Epoch: 0015 cost = 0.021565356
Learning Finished!!


"\ncorrect_prediction = tf.equal(tf.argmax(hypothesis, axis=1), tf.argmax(Y, axis=1))\naccuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))\nprint('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))\n"